In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import os

In [2]:
raw_data = pd.read_csv('data.csv')
raw_data

,Name,Week,Score Projected,Score Actual,Bench Projected,Bench Actual,QB Projected,QB Actual,RB1 Projected,RB1 Actual,...,WR2 Actual,TE Projected,TE Actual,FLEX Projected,FLEX Actual,D Projected,D Actual,K Projected,K Actual,Win
0,Didi,1,126.6,108.48,100.5,100.30,23.4,24.28,17.7,16.5,...,4.7,9.9,14.9,14.2,13.9,6.2,2,8.2,13,0
1,Luk,1,132.6,131.56,86.8,80.72,22.0,29.36,17.3,14.5,...,10.1,13.6,12.5,14.9,11.6,7.8,16,8.2,1,1
2,Derek,1,126.3,105.32,62.5,73.32,18.3,18.32,19.7,24.4,...,0.0,11.8,12.7,14.7,19.4,3.9,-7,8.0,15,1
3,Unit,1,111.6,87.12,63.4,47.36,16.5,8.72,17.5,12.0,...,8.0,15.3,15.3,12.9,9.3,5.9,8,8.3,11,0
4,Skyler,1,122.0,122.02,77.3,82.60,19.8,26.02,18.0,12.8,...,25.2,11.3,4.5,14.7,8.1,3.9,11,7.9,2,0
5,Birdy,1,127.7,123.96,88.7,105.80,20.8,38.76,15.4,17.9,...,6.3,11.3,6.7,19.0,12.1,6.5,5,8.2,3,1
6,Fraus,1,93.0,81.42,59.7,63.20,20.0,8.82,17.7,23.2,...,14.8,15.0,12.1,0.2,0.0,5.3,14,8.7,5,0
7,Sierra,1,105.6,86.90,57.8,38.70,15.3,10.90,18.4,15.0,...,9.3,7.6,12.5,10.7,16.2,5.9,2,8.5,7,1
8,Elijah,1,128.5,68.84,89.8,104.52,18.5,6.84,15.7,13.9,...,9.0,11.9,1.5,16.6,1.8,6.9,4,8.1,6,0
9,Zander,1,128.3,127.98,80.3,70.22,19.0,22.58,20.9,18.4,...,9.1,11.3,13.9,13.5,12.3,8.3,1,8.6,8,1


In [3]:
major_stats = ['Score','Bench','QB','RB1','RB2','WR1','WR2','TE','FLEX','D','K']

Week by Week stats

In [5]:
total_weeks = raw_data['Week'].max()

for week in range(1, total_weeks + 1):
    week_data = raw_data[raw_data['Week'] == week]
    print(f"processing week {week} ...")
    for stat in major_stats:
        print(f'Processing {stat}...')
        data = {
            "line_x": [],
            "line_y": [],
            "Predicted": [],
            "Actual": []
        }

        # I have no idea why all this is necessary, I'm just following the documentaion
        players = (
            week_data.sort_values(by=[f'{stat} Actual'], ascending=True)['Name'].unique()
        )

        marker_color_actual = []

        for player in players:
            pred = week_data.loc[week_data['Name'] == player][f'{stat} Projected'].values[0]
            act = week_data.loc[week_data['Name'] == player][f'{stat} Actual'].values[0]
            data["Predicted"].append(pred)
            data["Actual"].append(act)
            data["line_x"].extend([pred, act, None])
            data["line_y"].extend([player,player,None])
            marker_color_actual.append('green' if act > pred else 'crimson')

        fig = go.Figure(data = [
            go.Scatter(
                x = data["line_x"],
                y = data["line_y"],
                mode = 'markers+lines',
                showlegend = False,
                marker=dict(
                    symbol='arrow',
                    color='black',
                    angleref='previous',
                    standoff=8,
                    size=16
                )
            ),
            go.Scatter(
                x = data['Predicted'],
                y = players,
                mode = 'markers+text',
                name = 'Predicted',
                marker = dict (color='silver',size=16),
                text = [str(x) for x in data['Predicted']],
                textposition = 'bottom center',
                showlegend = False,
            ),
            go.Scatter(
                x = data['Actual'],
                y = players,
                mode = 'markers+text',
                name = 'Actual',
                marker = dict (color=marker_color_actual, size=16),
                text = [str(x) for x in data['Actual']],
                textposition = 'top center',
                showlegend = False,
            )
        ])
        fig.update_layout(
            title = f'Week {week} Fantasy Football {stat} Scores: Projected vs Actual',
            height = 75 * len(players)
        )
        os.makedirs(f'Graphs/weeks/{week}', exist_ok=True)
        fig.write_image(f'Graphs/weeks/{week}/{stat}.png',height=75 * len(players))

processing week 1 ...
Processing Score...
Processing Bench...
Processing QB...
Processing RB1...
Processing RB2...
Processing WR1...
Processing WR2...
Processing TE...
Processing FLEX...
Processing D...
Processing K...
processing week 2 ...
Processing Score...
Processing Bench...
Processing QB...
Processing RB1...
Processing RB2...
Processing WR1...
Processing WR2...
Processing TE...
Processing FLEX...
Processing D...
Processing K...
processing week 3 ...
Processing Score...
Processing Bench...
Processing QB...
Processing RB1...
Processing RB2...
Processing WR1...
Processing WR2...
Processing TE...
Processing FLEX...
Processing D...
Processing K...


High Level Overviews

In [8]:
power_points = {}
# Iterate every week
for week in range(1, total_weeks + 1):
    week_data = raw_data[raw_data['Week'] == week]
    power_points[week] = {}
    # Calculate power points
    for idx, row in week_data.iterrows():
        if week > 1:
            power_points[week][row['Name']] = power_points[week-1][row['Name']]
        else:
            power_points[week][row['Name']] = 0
        if row['Win']:
            power_points[week][row['Name']] += 10
        for stat in major_stats:
            # Prevents cases where a player is projected 0 but scores points
            if row[f'{stat} Projected'] == 0:
                power_points[week][row['Name']] += round(row[f'{stat} Actual'])
            else:
                power_points[week][row['Name']] += round(((row[f'{stat} Actual'] / (row[f'{stat} Projected'] + 0.001))-1)*10)
    for stat in major_stats:
        players = week_data.sort_values(by=[f'{stat} Actual'], ascending=False)['Name'].unique()
        for i, player in enumerate(players):
            power_points[week][player] += len(players) - i

In [9]:
weekly_info = pd.DataFrame(columns=['Name','Week','Power Points'])

In [10]:
for week in range(1, total_weeks + 1):
    players = raw_data['Name'].unique().tolist()
    for player in players:
        weekly_info.loc[len(weekly_info)] = [player, week, power_points[week][player]]

In [11]:
ranking =  weekly_info.sort_values(by='Power Points', ascending=False)[weekly_info['Week'] == total_weeks][['Name','Power Points']]
ranking

C:\Users\285545\AppData\Local\Temp\1\ipykernel_20688\3905459840.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,Name,Power Points
27,Sierra,235
24,Skyler,225
25,Birdy,212
22,Derek,210
20,Didi,195
28,Elijah,186
21,Luk,156
29,Zander,156
26,Fraus,115
23,Unit,79


In [15]:
fig = px.line(weekly_info, x='Week', y='Power Points',color='Name',title='Player Power Points Over Time',symbol='Name')
fig.show()
fig.write_image(f'Graphs/power_points.png',height=75 * len(players))

Just tracking the raw score week by week for each stat

In [16]:
os.makedirs(f'Graphs/overview/projected', exist_ok=True)
os.makedirs(f'Graphs/overview/actual', exist_ok=True)
os.makedirs(f'Graphs/overview/cumulative actual', exist_ok=True)
os.makedirs(f'Graphs/overview/cumulative projected', exist_ok=True)

for stat in major_stats:
    print(f'Processing {stat}...')
    dataset_projected = {}
    dataset_actual = {}
    dataset_cumulative = {}
    dataset_cumulative_projected = {}
    
    for week in range(1, total_weeks + 1):
        
        week_data = raw_data[raw_data['Week'] == week]
        dataset_projected[week] = {}
        dataset_actual[week] = {}
        dataset_cumulative[week] = {}
        dataset_cumulative_projected[week] = {}
        
        for idx, row in week_data.iterrows():
            if week > 1:
                dataset_cumulative[week][row['Name']] = dataset_cumulative[week-1][row['Name']]
                dataset_cumulative_projected[week][row['Name']] = dataset_cumulative_projected[week-1][row['Name']]
            else:
                dataset_cumulative[week][row['Name']] = 0
                dataset_cumulative_projected[week][row['Name']] = 0
            
            dataset_projected[week][row['Name']] = row[f'{stat} Projected']
            dataset_actual[week][row['Name']] = row[f'{stat} Actual']
            dataset_cumulative[week][row['Name']] += row[f'{stat} Actual']
            dataset_cumulative_projected[week][row['Name']] += row[f'{stat} Projected']
    
    weekly_info_projected = pd.DataFrame(columns=['Name','Week','Points'])
    weekly_info_actual = pd.DataFrame(columns=['Name','Week','Points'])
    weekly_info_cumulative = pd.DataFrame(columns=['Name','Week','Points'])
    weekly_info_cumulative_projected = pd.DataFrame(columns=['Name','Week','Points'])
    
    for week in range(1, total_weeks + 1):
        
        players = raw_data['Name'].unique().tolist()
        
        for player in players:
            weekly_info_projected.loc[len(weekly_info_projected)] = [player, week, dataset_projected[week][player]]
            weekly_info_actual.loc[len(weekly_info_actual)] = [player, week, dataset_actual[week][player]]
            weekly_info_cumulative.loc[len(weekly_info_cumulative)] = [player, week, dataset_cumulative[week][player]]
            weekly_info_cumulative_projected.loc[len(weekly_info_cumulative_projected)] = [player, week, dataset_cumulative_projected[week][player]]
    
    fig = px.line(weekly_info_projected, x='Week', y='Points',color='Name',title=f'Player {stat} Projected Over Time',symbol='Name')
    fig.update_traces(textposition='bottom right')
    fig.write_image(f'Graphs/overview/projected/{stat}.png',height=75 * len(players))
    
    fig = px.line(weekly_info_actual, x='Week', y='Points',color='Name',title=f'Player {stat} Actual Over Time',symbol='Name')
    fig.update_traces(textposition='bottom right')
    fig.write_image(f'Graphs/overview/actual/{stat}.png',height=75 * len(players))
    
    fig = px.line(weekly_info_cumulative, x='Week', y='Points',color='Name',title=f'Player {stat} Cumulative Actual Over Time',symbol='Name')
    fig.update_traces(textposition='bottom right')
    fig.write_image(f'Graphs/overview/cumulative actual/{stat}.png',height=75 * len(players))

    fig = px.line(weekly_info_cumulative_projected, x='Week', y='Points',color='Name',title=f'Player {stat} Cumulative Projected Over Time',symbol='Name')
    fig.update_traces(textposition='bottom right')
    fig.write_image(f'Graphs/overview/cumulative projected/{stat}.png',height=75 * len(players))

Processing Score...
Processing Bench...
Processing QB...
Processing RB1...
Processing RB2...
Processing WR1...
Processing WR2...
Processing TE...
Processing FLEX...
Processing D...
Processing K...
